In [15]:
# proper orthogonal decomposition (POD)
import os
import modred as mr
import xarray as xr
import numpy as np

data_dir = "/home/jonas/Documents/Thesis/OceanPlatformControl/data/drifter_data/forecasts/area1"
data = xr.open_dataset(os.path.join(data_dir, "cmems_mod_glo_phy_anfc_merged-uv_PT1H-i-2022-04-21T12_30_00Z-2022-04-21T12_30_00Z-2022-04-30T23_30_00Z.nc"))
data = data["utotal"].isel(depth=0)
data = data.values

# reshape data (flattened snapshots vs time -> 2d array)
data = data.T
data = data.reshape(-1, data.shape[-1])
print(f"Shape of input data: {data.shape} [flattened space x time snapshots]")

# test with random data first
num_vecs = 30
data = np.random.random((100, num_vecs))
print(data.shape)

# compute POD
num_modes = 5
POD_res = mr.compute_POD_arrays_snaps_method(data, list(mr.range(num_modes)))
modes = POD_res.modes
eigvals = POD_res.eigvals

print(modes.shape)

Shape of input data: (198961, 228) [flattened space x time snapshots]
(100, 30)
(100, 5)
